# Adaptive Circuit Constrction

Mizore provides a framework for adaptive VQE like what is described in [J. Chem. Theory Comput. 2020, 16, 2](https://pubs.acs.org/doi/abs/10.1021/acs.jctc.9b01084) and [Nat Commun 10, 3007 (2019)](https://www.nature.com/articles/s41467-019-10988-2), where the structure of the parameterized quantum circuits is also optimized, differing from traditional VQE which uses a fixed parameterized circuit and only varies the parameter. While providing better performance of convergence, adaptive method can also achieve certain objective with fewer quantum gates. We believe that adaptive circuit construction is a key method for near-term quantum applications.


## Introduction

In adaptive circuit construction algorithms, ansatz circuits are adaptively constructed by selecting and adding blocks from a predefined pool $\mathcal{E}$.

Specifically, in each iteration, we do the following steps
- Go over the blocks in $\mathcal{E}$;
- Calculate a score of every block based on certain criterion;
- Add the entangler with highest score to the circuit.

We believe circuit construction is important for *near-term* quantum computing because their adaptiveness can help them place the entangling gates in a more efficient way than the methods use fixed circuits. The two-qubit gate advantage of including adaptiveness has been shown in some works.

To design a adaptive circuit construction algorithm, one needs to define
- The blocks pool $\mathcal{E}$;
- The criterion for scoring the blocks;
- Where to add the new block.

In Mizore, we provide a general framework `CircuitConstrutor` for design such a framework. 

Including
- Handy block pool tools (See [BlockPools](BlockPools.ipynb) for details);
- Modules for efficiently and parallelly calculate usually used scores;
- Extendable framework for define the way blocks are updated.

## Basic Usage

To begin with, we show how to start a simple circuit construction, in which
- New blocks are added to the end of the circuit;
- The blocks score is based on the energy they can decrease by adding it to the end of the circuit;
- Energy descent gradient is calculated first and the blocks with high gradient will be selected to do a parameter optimization;
- The energy descent presented in the parameter optimization is defined to be the score of a block.

This construction can be easily carried out with the class `GreedyConstructor`. Here we start from construct the problem Hamiltonian and block pool.

In [ ]:
from CircuitConstructor import GreedyConstructor
from HamiltonianGenerator import make_example_LiH
from HamiltonianGenerator.FermionTransform import jordan_wigner
from PoolGenerator import BlockPool,quasi_imaginary_evolution_rotation_pool
# Generate the problem to solve
energy_obj=make_example_LiH()
# Generate the block pool
pool=BlockPool(quasi_imaginary_evolution_rotation_pool(energy_obj.hamiltonian))

A plain constructor can be constructed with default parameters as follows.

In [ ]:
# A plain constructor with 4 processor
constructor=GreedyConstructor(energy_obj,pool) 

Constructor with various properties can be constructed by specify the parameters as follows. (Select one to run)

In [ ]:
# A 5 processor constructor with a name
from ParallelTaskRunner import TaskManager
task_manager=TaskManager(n_processor=5,task_package_size=20)
constructor=GreedyConstructor(energy_obj,pool,task_manager=task_manager,project_name="LiH") 

In [ ]:
# A constructor with specified optimizer
from ParameterOptimizere import BasinhoppingOptimizer
optimizer=BasinhoppingOptimizer(random_initial=0.01,niter=5)
constructor=GreedyConstructor(energy_obj,pool,optimizer=optimizer)

In [ ]:
# A constructor with specified initial circuit
from Blocks import BlockCircuit, HartreeFockInitBlock
bc=BlockCircuit(6)
bc.add_block(HartreeFockInitBlock([0,1]))
constructor=GreedyConstructor(energy_obj,pool,init_circuit=bc)

The construction will not start immediately after build the constructor. Use `execute_construction` to start the run. Figures of energy descent and time used, as well as log of the run will be stored in the path /mizore_result/`project_name`_`time`

In [ ]:
constructor.execute_construction()

## Extensions

### Other constuctors

We also provide a different kind of constructor `FixedDepthSweepConstructor`, which update the blocks in the circuit in a *sweep* way. 
Specifically, that is, update the blocks in the circuit of indices from `sweep_start_position` to `n_max_block-1` again and agian after constructing a circuit with `n_max_block`blocks.

Our numerical experiments show that, in this way, the number of blocks needed to achieve certain accuracy can be reduced. We also conjecture that by doing so we can avoiding local minimum in the calculation.

The following is an example for conserving gates by sweeps. The sweeps make the calculation converges by just 6 blocks. In contrast, a plain construction needs 7 blocks.

In [ ]:
from CircuitConstructor import FixedDepthSweepConstructor
from HamiltonianGenerator import make_example_H2,get_reduced_energy_obj_with_HF_init
from HamiltonianGenerator.FermionTransform import get_parity_transform,make_transform_spin_separating
from PoolGenerator import BlockPool,quasi_imaginary_evolution_rotation_pool

# Generate a symmetry reduced problem Hamiltonian
transform = make_transform_spin_separating(get_parity_transform(8),8)
energy_obj = make_example_H2(basis="6-31g", fermi_qubit_transform=transform)
energy_obj=get_reduced_energy_obj_with_HF_init(energy_obj,[3,7])

pool=BlockPool(quasi_imaginary_evolution_rotation_pool(energy_obj.hamiltonian))

constructor=FixedDepthSweepConstructor(energy_obj,pool,n_max_block=10,sweep_start_position=1,no_global_optimization=True)
constructor.execute_construction()

### More than ground state energy

The circuit constructors in Mizore are design to process a general type of objectives, not only the energy. By defining subclasses of `Objective` and `Cost`, the users can implement adaptive construction for any objective for which a cost function can be defined. 

Here, we show how to use the circuit constructors to construct an autoencoder which can compress a 4-qubit mixed state into a 2-qubit mixed state.

In [ ]:
from Blocks import BlockCircuit
from HamiltonianGenerator.FermionTransform import jordan_wigner
from HamiltonianGenerator import make_example_LiH
from CircuitConstructor import GreedyConstructor
from PoolGenerator import BlockPool,quasi_imaginary_evolution_rotation_pool,all_rotation_pool
from Objective import PurityObjective
import pickle
# Construct the states to compress
circuit_list=[]
for path in ["LiH_1.5_test.bc","LiH_1.3_test.bc"]:
    with open("Objective/"+path, "rb") as f:
        circuit_list.append(pickle.load(f))

# Define an objective for make qubit 0,1 as pure as possible
purity_obj=PurityObjective(circuit_list,[0,1])

# Run the construction for purity
pool=BlockPool(all_rotation_pool(6,4))
constructor=GreedyConstructor(purity_obj,pool,max_n_iter=10,gradient_screening_rate=0.02,project_name="LiH_purity")
constructor.execute_construction()